# CNN

In [22]:
DATASET_FOLDER = "../discard_datasets/cnn/dataset"
TRAINING_DATA_FOLDER = DATASET_FOLDER + "/training_set"
TESTING_DATA_FOLDER = DATASET_FOLDER + "/test_set"
SIGLE_PREDICTION_FOLDER = DATASET_FOLDER + "/single_prediction"

MODEL_FILE = "../models/cnn_model.keras"
LOAD_PRETRAINED_MODEL = False

## Importing the Libraries

In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.__version__

'2.14.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [4]:
PRE_RESCALE = 1.0 / 255
PRE_SIZE = (64, 64)
PRE_BATCH_SIZE = 32

In [5]:
# image augmentation
train_datagen = ImageDataGenerator(
    rescale=PRE_RESCALE, shear_range=0.2, zoom_range=0.2, horizontal_flip=True
)

training_set = train_datagen.flow_from_directory(
    TRAINING_DATA_FOLDER,
    target_size=PRE_SIZE,
    batch_size=PRE_BATCH_SIZE,
    class_mode="binary",
)

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [6]:
test_datagen = ImageDataGenerator(rescale=PRE_RESCALE)
test_set = test_datagen.flow_from_directory(
    TESTING_DATA_FOLDER,
    target_size=PRE_SIZE,
    batch_size=PRE_BATCH_SIZE,
    class_mode="binary",
)

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [7]:
cnn = tf.keras.models.Sequential()

#### Step 1 - Convolution

In [8]:
cnn.add(
    tf.keras.layers.Conv2D(
        filters=32, kernel_size=3, activation="relu", input_shape=[64, 64, 3]
    )
)

#### Step 2 - Pooling

In [9]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [10]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu"))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

#### Step 3 - Flattening

In [11]:
cnn.add(tf.keras.layers.Flatten())

#### Step 4 - Full Connection

In [12]:
cnn.add(tf.keras.layers.Dense(units=128, activation="relu"))

#### Step 5 - Output Layer

In [13]:
cnn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [14]:
# import visualkeras

# visualkeras.layered_view(cnn)
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 32)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

## Part 3 - Training the CNN 

### Compiling the CNN

In [15]:
cnn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

### Training the CNN on the Training set and evaluating it on th Test set

In [16]:
if not LOAD_PRETRAINED_MODEL:
    cnn.fit(x=training_set, validation_data=test_set, epochs=25)
    cnn.save(MODEL_FILE)

Epoch 1/25
250/250 [==============================] - 17s 67ms/step - loss: 0.6660 - accuracy: 0.5994 - val_loss: 0.6282 - val_accuracy: 0.6685
Epoch 2/25
250/250 [==============================] - 17s 70ms/step - loss: 0.6006 - accuracy: 0.6793 - val_loss: 0.6543 - val_accuracy: 0.6270
Epoch 3/25
250/250 [==============================] - 17s 68ms/step - loss: 0.5646 - accuracy: 0.7050 - val_loss: 0.5190 - val_accuracy: 0.7550
Epoch 4/25
250/250 [==============================] - 17s 67ms/step - loss: 0.5264 - accuracy: 0.7297 - val_loss: 0.5224 - val_accuracy: 0.7530
Epoch 5/25
250/250 [==============================] - 17s 68ms/step - loss: 0.4979 - accuracy: 0.7519 - val_loss: 0.4730 - val_accuracy: 0.7755
Epoch 6/25
250/250 [==============================] - 17s 68ms/step - loss: 0.4732 - accuracy: 0.7710 - val_loss: 0.5064 - val_accuracy: 0.7650
Epoch 7/25
250/250 [==============================] - 17s 67ms/step - loss: 0.4548 - accuracy: 0.7856 - val_loss: 0.4715 - val_accuracy:

In [ ]:
# loading model
if LOAD_PRETRAINED_MODEL:
    cnn = tf.keras.models.load_model(MODEL_FILE)

## Part 4 - Making a Single Prediction

In [24]:
# See indices of classes

training_set.class_indices

{'cats': 0, 'dogs': 1}

In [28]:
import numpy as np
from keras.preprocessing import image

img = SIGLE_PREDICTION_FOLDER + "/cat_or_dog_1.jpg"
test_image = image.load_img(img, target_size=PRE_SIZE)

# convert image to array
test_image = image.img_to_array(test_image)
# add batch dimension
test_image = np.expand_dims(test_image, axis=0)


result = cnn.predict(test_image / 255.0)

prediction = ""
if result[0][0] > 0.5:
    prediction = "dog"
else:
    prediction = "cat"

print(prediction)

1/1 [==============================] - 0s 15ms/step
dog
